In [1]:
import sys
import torch as t
import torch
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
sys.path.append('/workspace/wilson/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from jaxtyping import Float
from typing import Union
from einops import repeat
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
from llc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

In [17]:
#path = '/workspace/models/2024-08-14_21-24-30_gapS_5_'
# path = '/workspace/models/2024-08-16_20-45-42_F11_MLP3_256'
#path = '/workspace/models/2024-08-16_22-14-00_F11_MLP3_256_wd2e-4'
path = '/workspace/models/2024-08-16_22-53-29_SL2_5_MLP3_256'
models, params = load_models(path)

In [18]:
data = GroupData(params)

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: smallgrp(120, 5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


In [19]:
loss_dict = test_loss(models[-1].to(device), data)

In [20]:
loss_dict['G0_loss'].min().item(), loss_dict['G0_loss'].argmin().item()

(0.05674691125750542, 19)

In [21]:
instance = loss_dict['G0_loss'].argmin().item()

In [22]:
model = models[-1][instance].to(device)
lneurons, rneurons = model.get_neurons()
unembed = model.unembedding.data.detach()
lneurons, rneurons, unembed = lneurons.squeeze(0).to('cpu'), rneurons.squeeze(0).to('cpu'), unembed.squeeze(0).to('cpu')

In [23]:
irreps = data.groups[0].get_real_irreps()

In [24]:
irreps.keys()

dict_keys(['1d-0', '4d-0', '4d-1', '3d-0', '3d-1', '4d-2', '8d-0', '5d-0', '12d-0'])

In [25]:
irrep_bases = dict()
for name, irrep in irreps.items():
    irrep = einops.rearrange(irrep, 'N d1 d2 -> N (d1 d2)')
    U, S, V = torch.svd(irrep)
    nonzero = S > 1e-5
    irrep_bases[name] = U[:,nonzero]

In [26]:
for k, v in irrep_bases.items():
    print(k, v.shape)

1d-0 torch.Size([120, 1])
4d-0 torch.Size([120, 4])
4d-1 torch.Size([120, 4])
3d-0 torch.Size([120, 9])
3d-1 torch.Size([120, 9])
4d-2 torch.Size([120, 16])
8d-0 torch.Size([120, 16])
5d-0 torch.Size([120, 25])
12d-0 torch.Size([120, 36])


In [27]:
lexpl = dict()
for name, basis in irrep_bases.items():
    proj = basis.T @ lneurons
    lexpl[name] = proj.norm(dim=0)**2 / lneurons.norm(dim=0)**2
    #lexpl[name] = lexpl[name].nan_to_num(-1.)

In [28]:
lexpl_total, lirrep = t.stack(list(lexpl.values()), dim=0).max(dim=0)

In [29]:
lexpl_total

tensor([0.9653, 0.9648, 1.0000, 0.9718, 1.0000, 0.9315, 0.9656, 1.0000, 0.8626,
           nan, 0.9999, 0.9543, 1.0000, 0.9564, 1.0000, 1.0000, 0.9662, 0.9694,
        0.9645, 1.0000, 2.6250, 1.0000, 0.9722, 0.9590, 0.9625, 1.0000, 0.9621,
           nan, 0.9601,    nan, 0.9646, 0.9591, 0.9687, 0.9668, 0.9626, 0.9636,
        0.9433, 0.9729, 0.9650, 0.9609, 0.9772, 0.9682, 0.9649, 0.9643, 1.0000,
        0.9648,    nan, 0.9649, 0.9610, 0.9643, 1.0000, 0.9721, 1.0000, 1.0000,
        0.9689, 1.0000, 0.9651, 0.9406, 0.9650, 0.9604, 0.9712, 0.9592, 0.9573,
           nan, 0.9654, 1.0000, 0.9693, 0.9617, 1.0000, 0.9703, 0.9664, 0.9633,
        1.0000, 0.9761, 0.9653, 0.9672,    nan, 0.9693, 0.9355, 1.0000, 0.9759,
        0.9647, 0.9614, 0.9702, 0.9654, 0.9698, 0.9664, 0.9657, 0.9691, 0.9681,
        0.9682, 0.9788, 1.0000, 0.9639, 0.9591, 0.9705, 1.0000, 0.9640, 0.9612,
        0.9739, 0.9572, 0.9691, 0.9675, 0.9685, 0.9566, 0.9462, 0.9761, 0.9674,
        0.9655, 0.9684, 0.9636, 0.9610, 

In [30]:
irrep_counts = defaultdict(lambda: 0)
for i in range(lexpl_total.shape[0]):
    if t.isnan(lexpl_total[i]):
        irrep_counts['dead'] += 1
    elif lexpl_total[i] < 0.95:
        irrep_counts['none'] += 1
    else:
        irrep_counts[list(lexpl.keys())[lirrep[i]]] += 1


In [31]:
irrep_counts

defaultdict(<function __main__.<lambda>()>,
            {'4d-1': 182, '4d-2': 52, 'none': 13, 'dead': 9})

In [43]:
irrep1_neurons = [i for i in range(lneurons.shape[-1]) if lexpl['4d-1'][i].item() > 0.95]

In [40]:
irrep2_neurons = [i for i in range(lneurons.shape[-1]) if lexpl['4d-2'][i].item() > 0.95]

In [ ]:
len(irrep1_neu

In [45]:
','.join(map(str, irrep1_neurons))

'0,1,3,6,11,13,16,17,18,20,22,23,24,26,28,30,31,32,33,34,35,37,38,39,40,41,42,43,45,47,48,49,51,54,56,58,59,60,61,62,64,66,67,69,70,71,73,74,75,77,80,81,82,83,84,85,86,87,88,89,90,91,93,94,95,97,98,99,100,101,102,103,104,106,107,108,109,110,111,112,113,114,115,116,117,120,121,122,123,124,125,126,127,129,130,131,133,134,135,137,138,140,141,145,146,147,150,152,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,179,180,181,182,183,184,185,186,188,189,192,194,195,196,199,201,202,203,204,206,208,212,216,218,219,220,221,223,224,225,227,229,230,231,232,233,234,236,237,238,239,240,241,242,246,247,248,250,253,255'

In [34]:
lexpl['4d-2']

tensor([0.0085, 0.0055, 1.0000, 0.0044, 1.0000, 0.0419, 0.0011, 1.0000, 0.1167,
           nan, 0.9999, 0.0250, 1.0000, 0.0179, 1.0000, 1.0000, 0.0071, 0.0021,
        0.0056, 1.0000, 0.0000, 1.0000, 0.0013, 0.0229, 0.0133, 1.0000, 0.0154,
           nan, 0.0056,    nan, 0.0052, 0.0068, 0.0052, 0.0074, 0.0039, 0.0024,
        0.0203, 0.0053, 0.0054, 0.0065, 0.0025, 0.0036, 0.0096, 0.0041, 1.0000,
        0.0076,    nan, 0.0074, 0.0061, 0.0063, 1.0000, 0.0051, 1.0000, 1.0000,
        0.0019, 1.0000, 0.0095, 0.0353, 0.0044, 0.0109, 0.0043, 0.0114, 0.0097,
           nan, 0.0058, 1.0000, 0.0033, 0.0050, 1.0000, 0.0020, 0.0033, 0.0045,
        1.0000, 0.0045, 0.0052, 0.0027,    nan, 0.0058, 0.0404, 1.0000, 0.0046,
        0.0038, 0.0118, 0.0034, 0.0057, 0.0043, 0.0084, 0.0108, 0.0047, 0.0011,
        0.0031, 0.0020, 1.0000, 0.0076, 0.0191, 0.0051, 1.0000, 0.0091, 0.0062,
        0.0031, 0.0133, 0.0046, 0.0059, 0.0068, 0.0184, 0.0243, 0.0018, 0.0051,
        0.0051, 0.0088, 0.0145, 0.0161, 

In [41]:
params

Parameters(instances=100, embed_dim=256, hidden_size=256, epochs=10000, batch_size=64, batched=False, activation='relu', weight_decay=0.001, lr=0.01, beta1=0.9, beta2=0.98, optimizer='adam', checkpoint=3, name='smallgrp(110, 1)', seed=42, group_string='smallgrp(110, 1)', intersect_frac=1.0, delta_frac=[0.0], train_frac=0.4, save_weights=False, load_weights='', wandb=True, thresh_grok=0.95, project='group generalization', model='MLP3', unembed_bias=False, init_func='kaiming_uniform', correct_embed=False)

In [44]:
lexpl['10d-0'][lexpl['10d-0'] > 0.9].min()

tensor(0.9998)